In [13]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os
import torch.optim.lr_scheduler as lr_scheduler

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cpu


In [15]:
df_path = r"D:\Machine Learning\weather_prediction\2023_weather_conditions_df.xlsx"

df = pd.read_excel(df_path, engine = "openpyxl")

In [16]:
df.head()

,Homérséklet,Páratartalom,Légnyomás,month_sin,month_cos,day_sin,day_cos,idokulonbseg
0,6.6,82,1021.3,0.5,0.5,0.207912,0.207912,15
1,6.4,81,1020.7,0.5,0.5,0.207912,0.207912,16
2,6.3,81,1020.5,0.5,0.5,0.207912,0.207912,19
3,6.1,86,1020.4,0.5,0.5,0.207912,0.207912,15
4,5.9,86,1020.5,0.5,0.5,0.207912,0.207912,15


In [17]:
df.shape

(64509, 8)

In [18]:
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df)

scaled_df

array([[0.17368421, 0.775     , 0.58230088, ..., 0.60452846, 0.60452846,
        0.0400534 ],
       [0.16842105, 0.7625    , 0.57168142, ..., 0.60452846, 0.60452846,
        0.04072096],
       [0.16578947, 0.7625    , 0.56814159, ..., 0.60452846, 0.60452846,
        0.04272363],
       ...,
       [0.22631579, 0.6625    , 0.43539823, ..., 0.60452846, 0.60452846,
        0.04339119],
       [0.22105263, 0.675     , 0.43362832, ..., 0.60452846, 0.60452846,
        0.0400534 ],
       [0.21315789, 0.675     , 0.43539823, ..., 0.60452846, 0.60452846,
        0.0400534 ]])

In [19]:
class WeatherDataset(Dataset):
    def __init__(self, x_data, y_data, time_diff_data):
        self.x_data = x_data
        self.y_data = y_data
        self.time_diff_data = time_diff_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx], self.time_diff_data[idx]

In [20]:
def create_sequences(data, input_length, output_length):

    sequences_x = []
    sequences_y = []
    sequences_time_diff = []
    for i in range(len(data)-input_length-output_length+1):
        sequences_x.append(data[i:i+input_length])
        sequences_y.append(data[i+input_length:i+input_length+output_length,0]) #It only should use the first column
        sequences_time_diff.append(data[i+input_length:i+input_length+output_length,-1]) #It only should use the last column

    return np.array(sequences_x), np.array(sequences_y), np.array(sequences_time_diff)


In [21]:
input_length = 700
output_length = 300

sequences_x, sequences_y, sequences_time_diff = create_sequences(scaled_df, input_length, output_length)

In [23]:
sequences_x.shape, sequences_y.shape, sequences_time_diff.shape

((63510, 700, 8), (63510, 300), (63510, 300))

In [24]:
class LSTM_weather(nn.Module):
    def __init__(self, input_dim, hidden_size, num_layers, seq_length, output_length):
        super(LSTM_weather, self).__init__()
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.seq_length = seq_length
        self.output_length = output_length

        self.lstm = nn.LSTM(input_dim, hidden_size, num_layers, batch_first = True)
        self.fc_1 = nn.Linear(hidden_size, 128)
        self.fc_2 = nn.Linear(128, 1)
        self.fc_time = nn.Linear(128, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        output, (hn, cn) = self.lstm(x, (h_0, c_0))

        out = output[:, -self.output_length:, :]
        out = self.relu(out)
        out = self.fc_1(out)
        out = self.relu(out)

        out_value = self.fc_2(out)
        out_time = self.fc_time(out)



        return out_value, out_time

In [25]:
def train_val_split(sequences_x, sequences_y, sequences_time_diff, val_percentage):
    val_size = int(val_percentage*len(sequences_x)/100)
    val_x, train_x = sequences_x[:val_size], sequences_x[val_size:]
    val_y, train_y = sequences_y[:val_size], sequences_y[val_size:]
    val_time_diff, train_time_diff = sequences_time_diff[:val_size], sequences_time_diff[val_size:]

    return train_x, val_x, train_y,  val_y, train_time_diff, val_time_diff


In [26]:
x_train, x_val, y_train, y_val, train_time_diff, val_time_diff = train_val_split(sequences_x, sequences_y, sequences_time_diff, 20)

In [27]:
train_dataset = WeatherDataset(torch.tensor(x_train, dtype = torch.float32).to(device), torch.tensor(y_train, dtype = torch.float32).to(device), torch.tensor(train_time_diff, dtype = torch.float32).to(device))
val_dataset = WeatherDataset(torch.tensor(x_val, dtype = torch.float32).to(device), torch.tensor(y_val, dtype = torch.float32).to(device), torch.tensor(val_time_diff, dtype = torch.float32).to(device))

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = False, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, drop_last=True)

In [28]:
input_dim = 8
hidden_size = 512
num_layers = 2
seq_length = 700
output_length = 300

model = LSTM_weather(input_dim, hidden_size, num_layers, seq_length, output_length).to(device)

criterion_value = nn.MSELoss()
criterion_time = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr = 0.005, weight_decay=1e-5)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=60)

In [29]:
import matplotlib.pyplot as plt
def plotting_loss(epochs, train_loss, val_loss, train_loss_value, val_loss_value):

  plt.subplot(2,1,1)
  plt.plot(epochs, train_loss, color = 'b', label = 'Training Loss')
  plt.plot(epochs, val_loss, color = 'g', label = 'Validation Loss')
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.title('Summed up loss over Epochs')
  plt.legend()
  plt.grid(True)

  plt.subplot(2,1,2)
  plt.plot(epochs, train_loss_value, color = 'b', label = 'Training Loss')
  plt.plot(epochs, val_loss_value, color = 'g', label = 'Validation Loss')
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.title('Value loss over Epochs')
  plt.legend()
  plt.grid(True)

  plt.tight_layout()
  plt.show()
  plt.savefig('D:/Machine Learning/weather_prediction/best_weights_model/second_model_loss_epoch_fig.png')


In [ ]:
num_epochs = 120

best_loss1 = float('inf')
best_loss2 = float('inf')


train_loss_value = []
val_loss_value = []
val_loss = []
train_loss = []
epochs = []

for epoch in range(num_epochs):
    epochs.append(epoch+1)


    train_loss_value1 = 0
    val_loss_value1 = 0
    train_loss1 = 0
    val_loss1 = 0

    model.train()
    for x_batch, y_batch, time_diff_batch in train_loader:

        optimizer.zero_grad()

        output_value, output_time_diff = model(x_batch)

        loss1 = criterion_value(output_value.squeeze(-1), y_batch)
        loss2 = criterion_time(output_time_diff.squeeze(-1), time_diff_batch)

        loss = loss1 + loss2
        loss.backward()

        optimizer.step()

        train_loss1 += loss.item()
        train_loss_value1 += loss1.item()

    train_loss1 /= len(train_loader) #taking the average
    train_loss.append(train_loss1)

    train_loss_value1 /= len(train_loader) #taking the average
    train_loss_value.append(train_loss_value1)

    model.eval()
    with torch.no_grad():
        for x_val_batch, y_val_batch, time_diff_val_batch in val_loader:

            output_value_val, output_time_diff_val = model(x_val_batch)

            loss_value_val = criterion_value(output_value_val.squeeze(-1), y_val_batch)
            loss_time_val = criterion_time(output_time_diff.squeeze(-1), time_diff_val_batch)

            loss_val = loss_value_val + loss_time_val
            val_loss1 += loss_val.item()
            val_loss_value1 += loss_value_val.item()

    val_loss1 /= len(val_loader) #taking the average
    val_loss.append(val_loss1)

    val_loss_value1 /= len(val_loader) #taking the average
    val_loss_value.append(val_loss_value1)


    scheduler.step()

    print(f'Epoch: {epoch+1}, Training loss: {train_loss1}, Validation loss: {val_loss1}')

    if val_loss1 < best_loss1 or val_loss_value1 < best_loss2:
      if val_loss1 < best_loss1:
        best_loss1 = val_loss1
      if val_loss_value1 < best_loss2:
        best_loss2 = val_loss_value1

      torch.save(model.state_dict(), f'D:/Machine Learning/weather_prediction/best_weights_model/second_try_model_weights_{epoch+1}.pth')

best_epoch1 = val_loss.index(max(val_loss)) + 1
best_epoch2 = val_loss_value.index(max(val_loss_value)) + 1

print(f"The best performing epoch by summed up loss: {best_epoch1}")
print(f"The best performing epoch by value loss: {best_epoch2}")


plotting_loss(epochs, train_loss, val_loss, train_loss_value, val_loss_value)
